In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df=pd.read_csv('/Users/dexter/Training/data.csv')

In [ ]:
df.head()

In [ ]:
df.shape

##### The unnamed column is useless. So, removing it.

In [ ]:
df.drop('Unnamed: 0',axis=1, inplace=True)

In [ ]:
df.head()

In [ ]:
df=df.sample(800000)
df.shape

In [ ]:
#save data file to parquet with compression
# df.to_parquet('/Users/dexter/Training/spotify_data_insights/data.parquet',compression='gzip')
#compress more
# df.to_parquet('/Users/dexter/Training/spotify_data_insights/data.parquet',engine='fastparquet',compression='gzip')

In [ ]:
df.info()

In [ ]:
df.describe()

#### **Choosing popularity as the target variable, the aim is to train a model that will be able to predict the poplularity of tracks based on available features.**

##### Dropping unnecessary columns that are not needed for training

In [ ]:
df.drop('track_id',axis=1,inplace=True)
df.drop('artist_name',axis=1,inplace=True)
df.drop('track_name',axis=1,inplace=True)
df.drop('mode',axis=1,inplace=True)
df.drop('time_signature',axis=1,inplace=True)

In [ ]:
df.head()

In [ ]:
#Boxplot
import seaborn as sns
sns.boxplot(data=df,x='popularity')
plt.show(sns)   

##### Do we exclude these outliers? are they important or necessary for the right prediction? If yes, how?

In [ ]:
np.where(df['popularity']>60)

##### Choosing a label encoder to convert the categorical column 'genre' to better fit our model.

In [ ]:
#Performing label encoding on genre column
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
df['genre']=le.fit_transform(df['genre'])

In [ ]:
df.info()

In [ ]:
#research on this
a=df.corr()
a['popularity'].sort_values(ascending=False)

In [ ]:
#plotting correlation matrix heatmap
import seaborn as sns
plt.figure(figsize=(20,20))
sns.heatmap(df.corr(),annot=True,cmap='coolwarm')
plt.show(sns)

In [ ]:
#barplot for poplularity increasing every year
plt.figure(figsize=(20,20))
sns.barplot(x='year',y='popularity',data=df)
plt.show(sns)

In [ ]:
plt.figure(figsize=(20,20))
sns.barplot(x='genre',y='popularity',data=df)
plt.show(sns)

In [ ]:
df.head()

In [ ]:
df['popularity'].max()

##### Splitting the data into train and test split of 80% and 20%

In [ ]:
#performing a train test split with popularity as the target variable
from sklearn.model_selection import train_test_split
X=df.drop('popularity',axis=1)
y=df['popularity']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

### **Training a Linear Regression model ⬇**

In [ ]:
#Training a linear regression model
from sklearn.linear_model import LinearRegression
lr=LinearRegression()
lr.fit(X_train,y_train)

#Predicting the popularity of the songs
y_pred=lr.predict(X_test)

#Evaluating the model
from sklearn.metrics import mean_squared_error
print('Mean Squared Error:',mean_squared_error(y_test,y_pred))

#RMSE for the linear regression model
print('Root Mean Squared Error:',mean_squared_error(y_test,y_pred,squared=False))

##### The model is not performing well as the mean squared error is very high

### **Training a Decision Tree Model ⬇**

In [ ]:
from sklearn.tree import DecisionTreeRegressor
dt=DecisionTreeRegressor()
dt.fit(X_train,y_train)

y_pred2=dt.predict(X_test)

#Evaluating the model
from sklearn.metrics import mean_squared_error

print('Mean Squared Error:',mean_squared_error(y_test,y_pred2))

#RMSE for the decision tree model
print('Root Mean Squared Error:',mean_squared_error(y_test,y_pred2,squared=False))

##### The decision tree model has a lower mean squared error than the linear regression model.
##### This means that the decision tree model is a better fit for the data than the linear regression model.
##### However, the rmse of 13 implies that the model is predicting 13 units away from the actual value.

### **Training a Random forest model ⬇**

In [ ]:
#Random Forest Regressor
from sklearn.ensemble import RandomForestRegressor
rf=RandomForestRegressor()
rf.fit(X_train,y_train)

y_pred3=rf.predict(X_test)

#Evaluating the model
from sklearn.metrics import mean_squared_error

print('Mean Squared Error:',mean_squared_error(y_test,y_pred3))
print('Root Mean Squared Error:',mean_squared_error(y_test,y_pred3,squared=False))

##### The random forest model is performing better than the others

#### **Checking the RMSE of the training data**

In [ ]:
y_pred_train=dt.predict(X_train)
print('Root Mean Squared Error:',np.sqrt(mean_squared_error(y_train,y_pred_train)))

##### The RMSE of the training set is less than the RMSE of the test set, which means that the model is overfitting

In [ ]:
#average value of the target variable
df['popularity'].mean()

##### Then again the mean of the target variable is 18.383 which is more than the RMSE of the random forest regressor model. So, can we say that the model is not overfitting?